## This notebook is an example of how to pipeline two models. 
This notebook is an example of how to use DeGirum PySDK to do AI inference of a graphical file using 
two AI models: license plate detection and license recognition. The license plate detection model 
is run on the image and the results are then processed by the license recognition model, 
one license plate at a time. Combined result is then displayed.
This script uses PIL as image processing backend.

This script works with the following inference options:

1. Run inference on DeGirum Cloud Platform;
2. Run inference on DeGirum AI Server deployed on a localhost or on some computer in your LAN or VPN;
3. Run inference on DeGirum ORCA accelerator directly installed on your computer.

To try different options, you just need to uncomment **one** of the lines in the code below.

You also need to specify your cloud API access token, cloud zoo URLs, and AI server hostname in [env.ini](../../env.ini) file, located in the same directory as this notebook.

In [ ]:
# make sure degirum-tools package is installed
!pip show degirum-tools || pip install degirum-tools

#### Specify where do you want to run your inferences

In [43]:
# hw_location: Where you want to run inference.
#     Use "@cloud" to use DeGirum cloud.
#     Use "@local" to run on local machine.
#     Use an IP address for AI server inference.
hw_location = "@cloud"

# face_model_zoo_url: URL/path for the face model zoo.
#     Use cloud_zoo_url for @cloud, @local, and AI server inference options.
#     Use '' for an AI server serving models from a local folder.
#     Use a path to a JSON file for a single model zoo in case of @local inference.
face_model_zoo_url = "https://cs.degirum.com/degirum/ultralytics_v6"

# face_model_name: Name of the model for face detection.
face_model_name = "yolov8n_relu6_face--640x640_quant_n2x_orca1_1"

# gender_model_zoo_url: URL/path for the gender model zoo.
gender_model_zoo_url = "https://cs.degirum.com/degirum/openvino"

# gender_model_name: Name of the model for gender detection.
gender_model_name = "mobilenet_v2_gender--160x160_float_openvino_cpu_1"

# video_source: Video source for inference.
#     Use a camera index for a local webcam.
#     Use a URL of an RTSP stream.
#     Use a URL of a YouTube video.
#     Use a path to a video file (like an MP4).
video_source='../../images/faces_and_gender.mp4'                              

#### The rest of the cells below should run without any modifications

In [44]:
import degirum as dg
import degirum_tools

# Configure for Google Colab
degirum_tools.configure_colab()

# Connect to AI inference engine getting token from env.ini file
face_zoo = dg.connect(hw_location, face_model_zoo_url, degirum_tools.get_token())
gender_zoo = dg.connect(hw_location, gender_model_zoo_url, degirum_tools.get_token())

# Load models
with face_zoo.load_model(face_model_name) as face_model:
    with gender_zoo.load_model(gender_model_name) as gender_model:
        # Create a compound cropping model with 50% crop extent
        crop_model = degirum_tools.CroppingAndClassifyingCompoundModel(
            face_model, gender_model, 50.0
        )

        # Detect faces and genders
        with degirum_tools.Display("Faces and Gender") as display:
            for gender_results in degirum_tools.predict_stream(crop_model, video_source):
                display.show(gender_results)

Successfully opened video stream '../../images/faces_and_gender.mp4'
